In [39]:
import glob
import torch

from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [43]:
%run Data-Preprocessing.ipynb
%run U-Net.ipynb
%run ../Train-Eval-Utils.ipynb

Load model:

In [49]:
model = UNet(depth=3, in_channel=3)
model.load_state_dict(torch.load('models/modelLungsMultiSlices.pth'))
model.eval()

UNet(
  (down_blocks): ModuleList(
    (0): _DoubleConv(
      (conv): Sequential(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): ReLU(inplace=True)
        (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): _DownConv(
      (down): Sequential(
        (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (1): _DoubleConv(
          (conv): Sequential(
            (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): ReLU(inplace=True)
            (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
       

Load data for one scan an create loader (we assuming that slices are alredy provided):

In [32]:
test_dir = 'testScan'

In [35]:
convert_ct_scan_to_slices("lung_001.nii.gz", target_dir = test_dir+'/images', normalize=True)

In [37]:
convert_ct_scan_to_slices("lungLabel_001.nii.gz", target_dir = test_dir+'/labels', normalize=False)

In [40]:
transform = transforms.Compose([
    transforms.Resize((128, 128), antialias=False),
    transforms.ConvertImageDtype(torch.float)
])

test_dataset = CTDatasetMultiSlices(root_dir=test_dir, image_transform=transform, label_transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=os.cpu_count())

In [56]:
model.eval()
with torch.no_grad():
    for images, masks in test_loader:
        outputs = model(images)

In [57]:
# outputs

tensor([[[[2.2354e-06, 1.0096e-07, 1.2180e-07,  ..., 2.2341e-07,
           2.1203e-07, 1.1844e-06],
          [3.5825e-07, 3.4550e-08, 4.2903e-08,  ..., 1.5249e-07,
           1.2946e-07, 1.4224e-07],
          [1.9162e-07, 1.3935e-08, 1.6089e-08,  ..., 1.0199e-07,
           1.0170e-07, 1.1693e-07],
          ...,
          [2.8563e-07, 2.2321e-08, 2.7657e-08,  ..., 5.6698e-08,
           7.6134e-08, 6.5649e-08],
          [4.5893e-07, 4.5181e-08, 4.6478e-08,  ..., 1.4932e-07,
           1.7764e-07, 1.1673e-07],
          [9.5646e-07, 7.7654e-08, 6.4452e-08,  ..., 1.0287e-07,
           1.3066e-07, 5.4609e-07]]],


        [[[2.2499e-06, 1.0145e-07, 1.1836e-07,  ..., 2.1626e-07,
           2.0000e-07, 1.2400e-06],
          [3.6324e-07, 3.5633e-08, 4.1644e-08,  ..., 1.4491e-07,
           1.0612e-07, 1.4491e-07],
          [1.9445e-07, 1.4675e-08, 1.6697e-08,  ..., 1.0317e-07,
           8.4706e-08, 1.2153e-07],
          ...,
          [2.4512e-07, 2.1216e-08, 2.8572e-08,  ..., 5.88

In [62]:
createGifFromModelOutputs(outputs, "model_output_animation")

In [75]:
folder_path = test_dir + '/images/'

# Get a list of all files in the folder that start with '1_'
file_names = [folder_path + f for f in os.listdir(folder_path) if f.endswith('.png')]

# Sort filenames to ensure correct order (optional, depending on naming convention)
file_names = sorted(file_names, key=numerical_sort)
# file_names

createGifFromSlices(paths = file_names, gif_name = 'scan1')

In [76]:
folder_path = test_dir + '/labels/'

# Get a list of all files in the folder that start with '1_'
file_names = [folder_path + f for f in os.listdir(folder_path) if f.endswith('.png')]

# Sort filenames to ensure correct order (optional, depending on naming convention)
file_names = sorted(file_names, key=numerical_sort)
# file_names

createGifFromSlices(paths = file_names, gif_name = 'scan2')